In [ ]:
!pip3 install pyspark
import pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 51.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9114b10f3c15d406cb3850cb98cfa71a30733d3894f73338cb8d6f2a0d71eb2f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
spark = pyspark.sql.SparkSession.builder\
        .master('local[*]')\
        .appName('vikram_hw2 final')\
        .getOrCreate()

In [ ]:
sc = spark.sparkContext
sc.setLogLevel("ERROR")
sc

<SparkContext master=local[*] appName=vikram_hw2 final>

# Question 1

In [ ]:
# question 1
from pyspark.sql.functions import col,isnan,when,count
data = spark.read.csv('BreadBasket_DMS.csv', header=True)
data.show()

+----------+--------+-----------+-------------+
|      Date|    Time|Transaction|         Item|
+----------+--------+-----------+-------------+
|2016-10-30|09:58:11|          1|        Bread|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:07:57|          3|Hot chocolate|
|2016-10-30|10:07:57|          3|          Jam|
|2016-10-30|10:07:57|          3|      Cookies|
|2016-10-30|10:08:41|          4|       Muffin|
|2016-10-30|10:13:03|          5|       Coffee|
|2016-10-30|10:13:03|          5|       Pastry|
|2016-10-30|10:13:03|          5|        Bread|
|2016-10-30|10:16:55|          6|    Medialuna|
|2016-10-30|10:16:55|          6|       Pastry|
|2016-10-30|10:16:55|          6|       Muffin|
|2016-10-30|10:19:12|          7|    Medialuna|
|2016-10-30|10:19:12|          7|       Pastry|
|2016-10-30|10:19:12|          7|       Coffee|
|2016-10-30|10:19:12|          7|          Tea|
|2016-10-30|10:20:51|          8|       

In [ ]:
from pyspark.sql.window import Window
import pyspark.sql.functions as f
data=data.select(data.Date,data.Time[0:2],data.Transaction, data.Item )
data=data.withColumnRenamed("substring(Time, 0, 2)","Time")
data.show()
# naturalising hours

+----------+----+-----------+-------------+
|      Date|Time|Transaction|         Item|
+----------+----+-----------+-------------+
|2016-10-30|  09|          1|        Bread|
|2016-10-30|  10|          2| Scandinavian|
|2016-10-30|  10|          2| Scandinavian|
|2016-10-30|  10|          3|Hot chocolate|
|2016-10-30|  10|          3|          Jam|
|2016-10-30|  10|          3|      Cookies|
|2016-10-30|  10|          4|       Muffin|
|2016-10-30|  10|          5|       Coffee|
|2016-10-30|  10|          5|       Pastry|
|2016-10-30|  10|          5|        Bread|
|2016-10-30|  10|          6|    Medialuna|
|2016-10-30|  10|          6|       Pastry|
|2016-10-30|  10|          6|       Muffin|
|2016-10-30|  10|          7|    Medialuna|
|2016-10-30|  10|          7|       Pastry|
|2016-10-30|  10|          7|       Coffee|
|2016-10-30|  10|          7|          Tea|
|2016-10-30|  10|          8|       Pastry|
|2016-10-30|  10|          8|        Bread|
|2016-10-30|  10|          9|   

In [ ]:
data = data.withColumn('Transaction', f.count('*').over(Window.partitionBy(f.to_date(data.Date), data.Item, data.Time)))
data = data.distinct()
data = data.sort(data.Date,data.Time)
data.show()
# all distinct elements per hour only final record

+----------+----+-----------+-------------+
|      Date|Time|Transaction|         Item|
+----------+----+-----------+-------------+
|2016-10-30|  09|          1|        Bread|
|2016-10-30|  10|          1|       Basket|
|2016-10-30|  10|          2|   Farm House|
|2016-10-30|  10|          1|        Fudge|
|2016-10-30|  10|          3|          Jam|
|2016-10-30|  10|          3|         NONE|
|2016-10-30|  10|          2|Hot chocolate|
|2016-10-30|  10|          1|      Cookies|
|2016-10-30|  10|          6| Scandinavian|
|2016-10-30|  10|         12|        Bread|
|2016-10-30|  10|          6|       Pastry|
|2016-10-30|  10|          7|       Coffee|
|2016-10-30|  10|          1|Mineral water|
|2016-10-30|  10|          1|      Tartine|
|2016-10-30|  10|          2|          Tea|
|2016-10-30|  10|          4|       Muffin|
|2016-10-30|  10|          8|    Medialuna|
|2016-10-30|  11|          1|       Basket|
|2016-10-30|  11|          7|        Bread|
|2016-10-30|  11|          4|   

In [ ]:
# name change to number of transanctions
data = data.select(data.Item, data.Time, data.Transaction.alias("Number of items"))

In [ ]:
data = data.groupBy('Item','Time').sum('Number of items')\
.withColumnRenamed('sum(Number of items)', 'total')

In [ ]:
data = data.filter(data.Time >= 9)
data = data.filter(data.Time < 23)

In [ ]:
data = data.select(data.Item, data.total)

In [ ]:
data.show()

+--------------------+-----+
|                Item|total|
+--------------------+-----+
|               Scone|   32|
|            Baguette|   12|
|           The Nomad|   11|
|            Mortimer|    1|
|Raspberry shortbr...|    1|
|      Dulce de Leche|    1|
|               Bread|  400|
|    Valentine's card|    2|
|        Scandinavian|   17|
|   Gingerbread syrup|    3|
|      Jammie Dodgers|   17|
|                Eggs|    5|
|                Eggs|    1|
|        Scandinavian|   58|
|             Cookies|   77|
|Extra Salami or Feta|    3|
|           Medialuna|  120|
|               Scone|   50|
|    Keeping It Local|    3|
|   Lemon and coconut|    3|
+--------------------+-----+
only showing top 20 rows



In [ ]:
data = data.groupby('Item').sum('total').withColumnRenamed('sum(total)', 'total')

In [ ]:
from pyspark.sql.functions import desc
data.sort(desc('total')).show(5)

+------+-----+
|  Item|total|
+------+-----+
|Coffee| 5259|
| Bread| 3151|
|   Tea| 1414|
|  Cake| 1017|
|Pastry|  797|
+------+-----+
only showing top 5 rows



# Question 2

In [ ]:
#question 2
data = spark.read.option("delimiter", ";").csv("Restaurants_in_Durham_County_NC.csv", header = True)

In [ ]:
data.show(5)
data.printSchema()
data.count()

+-----+--------------------+--------------------+----------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+--------------------+----------------+------+----------------------+--------------------+
|   ID|        Premise_Name|    Premise_Address1|Premise_Address2|Premise_City|Premise_State|Premise_Zip| Premise_Phone|Hours_Of_Operation|Opening_Date|Closing_Date|Seats|               Water|              Sewage|Insp_Freq|      Est_Group_Desc|Risk|Smoking_Allowed|    Type_Description|   Rpt_Area_Desc|Status|Transitional_Type_Desc|         geolocation|
+-----+--------------------+--------------------+----------------+------------+-------------+-----------+--------------+------------------+------------+------------+-----+--------------------+--------------------+---------+--------------------+----+---------------+--------------------+----

2472

In [ ]:
#total items are 2472 rows
data = data.withColumn('new_column', f.lit(0))

In [ ]:
data = data.select(data.Rpt_Area_Desc,data.new_column.alias("Count"))

In [ ]:
data = data.withColumn('Count', f.count('*').over(Window.partitionBy(data.Rpt_Area_Desc, data.Count)))
# using window we get only required features

In [ ]:
data = data.distinct()
# used to remove all the same repeating elements

In [ ]:
data.show()

+--------------------+-----+
|       Rpt_Area_Desc|Count|
+--------------------+-----+
|        Food Service| 1093|
|   Local Confinement|    2|
|         Summer Food|  242|
|      Swimming Pools|  420|
|        Institutions|   30|
|            Day Care|  173|
|         Mobile Food|  147|
|        Summer Camps|    4|
|   Bed&Breakfast Inn|    2|
|    School Buildings|   89|
|  Bed&Breakfast Home|    4|
|Tattoo Establishm...|   32|
|                null|   13|
|             Lodging|   62|
|    Residential Care|  154|
|      Adult Day Care|    5|
+--------------------+-----+



In [ ]:
data.groupby().sum().show()
# proof that initial count is equal to final

+----------+
|sum(Count)|
+----------+
|      2472|
+----------+



In [ ]:
data = data.filter(data.Rpt_Area_Desc!="null")
data.show()

+--------------------+-----+
|       Rpt_Area_Desc|Count|
+--------------------+-----+
|        Food Service| 1093|
|   Local Confinement|    2|
|         Summer Food|  242|
|      Swimming Pools|  420|
|        Institutions|   30|
|            Day Care|  173|
|         Mobile Food|  147|
|        Summer Camps|    4|
|   Bed&Breakfast Inn|    2|
|    School Buildings|   89|
|  Bed&Breakfast Home|    4|
|Tattoo Establishm...|   32|
|             Lodging|   62|
|    Residential Care|  154|
|      Adult Day Care|    5|
+--------------------+-----+



# Question 3

In [ ]:
# question 3
data = spark.read.option("delimiter",",").csv('populationbycountry19802010millions.csv', header=True)


In [ ]:
import pyspark.sql.functions as f

data2 = data.select(
    '_c0',
    *[((f.col(str(year)) - f.col(str(year-1))) / f.col(str(year-1))).alias('%s'%year)
      for year in range(1981,2011)]
)
#function to convert the columns into percent values

In [ ]:
import pandas as pd

In [ ]:
data1 = data2.toPandas()
data1 = data1.melt(id_vars=["_c0"],
        var_name="Year",
        value_name="Value")

data1 = data1.sort_values(["_c0","Year"])
print(data1)
#converting to numpy to melt no other alternative for spark was found

              _c0  Year     Value
185   Afghanistan  1981 -0.091063
417   Afghanistan  1982 -0.080172
649   Afghanistan  1983 -0.011676
881   Afghanistan  1984  0.026005
1113  Afghanistan  1985  0.026648
...           ...   ...       ...
5983     Zimbabwe  2006 -0.008174
6215     Zimbabwe  2007 -0.008761
6447     Zimbabwe  2008 -0.008134
6679     Zimbabwe  2009  0.003746
6911     Zimbabwe  2010  0.022754

[6960 rows x 3 columns]


In [ ]:
data1= spark.createDataFrame(data1) 
# converting back to spark

In [ ]:
data1.show()

+-----------+----+--------------------+
|        _c0|Year|               Value|
+-----------+----+--------------------+
|Afghanistan|1981|-0.09106330931425992|
|Afghanistan|1982|-0.08017227257036874|
|Afghanistan|1983|-0.01167567619139...|
|Afghanistan|1984|0.026005222604254943|
|Afghanistan|1985| 0.02664844470371272|
|Afghanistan|1986|-6.68259798025093...|
|Afghanistan|1987|-0.00653192673114...|
|Afghanistan|1988|0.007256438521818884|
|Afghanistan|1989| 0.02392884582192217|
|Afghanistan|1990|0.003149814203476...|
|Afghanistan|1991|0.005784657571321175|
|Afghanistan|1992| 0.09817500630213119|
|Afghanistan|1993| 0.13224594754698688|
|Afghanistan|1994| 0.08727661664211225|
|Afghanistan|1995|0.051303853790332456|
|Afghanistan|1996|0.033221262523444865|
|Afghanistan|1997| 0.03149161953520448|
|Afghanistan|1998|  0.0313271404601805|
|Afghanistan|1999|0.032625690181581436|
|Afghanistan|2000|0.009136831126266252|
+-----------+----+--------------------+
only showing top 20 rows



In [ ]:
data1 = data1.filter(data1.Value!="NaN")
#removing all nan values

In [ ]:
data1.sort(data1.Value.desc()).show()
#final answer

+--------------------+----+-------------------+
|                 _c0|Year|              Value|
+--------------------+----+-------------------+
|              Kuwait|1992| 0.4863343882962002|
|Falkland Islands ...|1997| 0.2149999999999999|
|              Rwanda|1996| 0.1961417728550077|
|          Montserrat|2000|0.16863905325443793|
|              Rwanda|1997|0.16817638846032829|
|       French Guiana|1983|0.14285714285714277|
|          Montserrat|2002|  0.134433962264151|
|         Afghanistan|1993|0.13224594754698688|
|            Djibouti|1990| 0.1282404791501865|
|          Montserrat|2008|0.12638580931263863|
|               Qatar|1983| 0.1263433611956396|
|       French Guiana|1985|0.12499999999999993|
|      Western Sahara|1981|0.12133182844243788|
|             Liberia|1998|0.12017449760423381|
|              Jordan|1991|0.11273939557210028|
|      Western Sahara|1982|0.11115105327485802|
|       French Guiana|1987|0.11111111111111122|
|      Cayman Islands|1988|0.11010421386

# Question 4

In [ ]:
text = sc.textFile("romeo-juliet-pg1777.txt")
# note as text file

In [ ]:
import re
def func(text):
    text = text.lower()
    text = re.sub("[^0-9a-zA-Z]", " ", text)
    return text
text1 = text.map(func)
#function similar to assignment 1 
# converted to lower and replaced all special characters

In [ ]:
text2 = text1.flatMap(lambda l: l.split(" "))
# split on white spaces

In [ ]:
from operator import add
count = text2.map(lambda w: (w, 1)).reduceByKey(add)
for i in count.take(10): print(i)

('', 22916)
('this', 258)
('is', 389)
('presented', 1)
('project', 27)
('gutenberg', 24)
('in', 329)
('cooperation', 1)
('world', 32)
('library', 16)


In [ ]:
final = count.toDF(schema=['word', 'count']).coalesce(8)
final.show(10)
# total count is 22916

+-----------+-----+
|       word|count|
+-----------+-----+
|           |22916|
|       this|  258|
|         is|  389|
|  presented|    1|
|    project|   27|
|  gutenberg|   24|
|         in|  329|
|cooperation|    1|
|      world|   32|
|    library|   16|
+-----------+-----+
only showing top 10 rows



In [ ]:
final=final.filter(final.word!="")
final.show(10)
# final answer

+-----------+-----+
|       word|count|
+-----------+-----+
|       this|  258|
|         is|  389|
|  presented|    1|
|    project|   27|
|  gutenberg|   24|
|         in|  329|
|cooperation|    1|
|      world|   32|
|    library|   16|
|        inc|   11|
+-----------+-----+
only showing top 10 rows

